# Preprocessing: LL19


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [3]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns

In [4]:
## Prep paths ##

subject = 'LL19'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [5]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/LL19/ieeg/LL19_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 784059 =      0.000 ...  1531.365 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/LL19/ieeg/LL19_raw_clean_ieeg.fif...
    Range : 0 ... 784059 =      0.000 ...  1531.365 secs
Ready.


In [7]:
## Channels did not include probe prefixes in bci data file, convert to descriptive names here

# read in csv with correct names
channel_csv = pd.read_csv(f"{preproc_data_dir}/../LL19_channel_names.csv")

# convert to dict
channel_dict = dict(zip(channel_csv.bci_names, channel_csv.use_names))

# rename
mne.rename_channels(filtered_clean_fif.info, channel_dict)
mne.rename_channels(raw_clean_fif.info, channel_dict)


## Bipolar Rereferencing

In [8]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [9]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

RA1 noref
RA2 RA3
RA3 RA4
RA4 RA5
RA5 RA6
RA7 noref
RA8 noref
RA9 noref
RA10 noref
RH1 RH2
RH2 RH3
RH3 RH4
RH4 RH5
RH6 noref
RH7 noref
RH8 noref
RHP1 noref
RHP2 noref
RHP3 noref
RHP4 noref
RHP5 noref
RHP6 noref
RHP7 RHP8
RPC1 RPC2
RPC2 RPC3
RPC3 RPC4
RPC4 RPC5
RPC5 RPC6
RPC6 RPC7
RPC7 RPC8
RPC8 RPC9
RPC10 noref
RtPar1 RtPar2
RtPar2 RtPar3
RtPar3 RtPar4
RtPar4 RtPar5
RtPar5 RtPar6
RtPar6 RtPar7
RtPar7 RtPar8
RAC1 RAC2
RAC2 RAC3
RAC3 RAC4
RAC4 RAC5
RAC5 RAC6
RAC6 RAC7
RAC7 RAC8
ROF1 ROF2
ROF2 ROF3
ROF3 ROF4
ROF4 ROF5
ROF5 ROF6
ROF6 ROF7
ROF7 ROF8
ROF8 ROF9
ROF9 ROF10
ROF10 ROF11
ROF11 ROF12
LOF1 LOF2
LOF2 LOF3
LOF3 LOF4
LOF4 LOF5
LOF5 LOF6
LOF6 LOF7
LOF7 LOF8
LOF8 LOF9
LOF9 LOF10
EKG noref
CH75 noref
CH76 noref
CH77 noref
CH78 noref
CH79 noref
CH80 noref
CH81 noref
CH82 noref
CH83 noref
CH84 noref
CH85 noref
CH86 noref
CH87 noref
CH88 noref
CH89 noref
CH90 noref
CH91 noref
CH92 noref
CH93 noref
CH94 noref
CH95 noref
CH96 noref
CH97 noref
CH98 noref
CH99 noref
CH100 noref
CH101 noref
CH10

In [10]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 784059  =      0.000 ...  1531.365 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=51, n_times=784060
    Range : 0 ... 784059 =      0.000 ...  1531.365 secs
Ready.
Added the following bipolar channels:
RA2-RA3, RA3-RA4, RA4-RA5, RA5-RA6, RH1-RH2, RH2-RH3, RH3-RH4, RH4-RH5, RHP7-RHP8, RPC1-RPC2, RPC2-RPC3, RPC3-RPC4, RPC4-RPC5, RPC5-RPC6, RPC6-RPC7, RPC7-RPC8, RPC8-RPC9, RtPar1-RtPar2, RtPar2-RtPar3, RtPar3-RtPar4, RtPar4-RtPar5, RtPar5-RtPar6, RtPar6-RtPar7, RtPar7-RtPar8, RAC1-RAC2, RAC2-RAC3, RAC3-RAC4, RAC4-RAC5, RAC5-RAC6, RAC6-RAC7, RAC7-RAC8, ROF1-ROF2, ROF2-ROF3, ROF3-ROF4, ROF4-ROF5, ROF5-ROF6, ROF6-ROF7, ROF7-ROF8, ROF8-ROF9, ROF9-ROF10, ROF10-ROF11, ROF11-ROF12, LOF1-LOF2, LOF2-LOF3, LOF3-LOF4, LOF4-LOF5, LOF5-LOF6, LOF6-LOF7, LOF7-LOF8, LOF8-LOF9, LOF9-LOF10
Writing /home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif


/tmp/ipykernel_6093/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif
[done]
Reading 0 ... 784059  =      0.000 ...  1531.365 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=51, n_times=784060
    Range : 0 ... 784059 =      0.000 ...  1531.365 secs
Ready.
Added the following bipolar channels:
RA2-RA3, RA3-RA4, RA4-RA5, RA5-RA6, RH1-RH2, RH2-RH3, RH3-RH4, RH4-RH5, RHP7-RHP8, RPC1-RPC2, RPC2-RPC3, RPC3-RPC4, RPC4-RPC5, RPC5-RPC6, RPC6-RPC7, RPC7-RPC8, RPC8-RPC9, RtPar1-RtPar2, RtPar2-RtPar3, RtPar3-RtPar4, RtPar4-RtPar5, RtPar5-RtPar6, RtPar6-RtPar7, RtPar7-RtPar8, RAC1-RAC2, RAC2-RAC3, RAC3-RAC4, RAC4-RAC5, RAC5-RAC6, RAC6-RAC7, RAC7-RAC8, ROF1-ROF2, ROF2-ROF3, ROF3-ROF4, ROF4-ROF5, ROF5-ROF6, ROF6-ROF7, ROF7-ROF8, ROF8-ROF9, ROF9-ROF10, ROF10-ROF11, ROF11-ROF12, LOF1-LOF2, LOF2-LOF3, LOF3-LOF4, LOF4-LOF5, LOF5-LOF6, LOF6-LOF7, LOF7-LOF8, LOF8-LOF9, LOF9-LOF10
Writing /home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp

/tmp/ipykernel_6093/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_raw_clean_data.fif
[done]


In [14]:
# Visualize it #
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')
bp_filtered_fif.plot(events=events, color='b', bad_color = 'cyan', n_channels = 70, clipping = None, event_color = 'r')

Opening raw data file /home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif...
    Range : 0 ... 784059 =      0.000 ...  1531.365 secs
Ready.
480 events found
Event IDs: [0 1]


/tmp/ipykernel_6093/1103936875.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


/home/brooke/mambaforge/envs/ieeg_analysis2/lib/python3.9/site-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/brooke/mambaforge/envs/ieeg_analysis2/lib/python3.9/site-packages/scipy/stats/_stats_py.py:3146: RuntimeWarning: invalid value encountered in subtract
  z = (scores - mn) / std
/home/brooke/mambaforge/envs/ieeg_analysis2/lib/python3.9/site-packages/mne_qt_browser/_pg_figure.py:4338: RuntimeWarning: invalid value encountered in subtract
  self.mne.data = self.mne.data - np.nanmean(


Channels marked as bad:
['CH100', 'CH101', 'CH102', 'CH103', 'CH104', 'CH105', 'CH106', 'CH107', 'CH108', 'CH109', 'CH110', 'CH111', 'CH112', 'CH113', 'CH114', 'CH115', 'CH116', 'CH117', 'CH118', 'CH119', 'CH120', 'CH121', 'CH122', 'CH123', 'CH124', 'CH125', 'CH126', 'CH127', 'CH128', 'CH129', 'CH130', 'CH131', 'CH132', 'CH133', 'CH134', 'CH135', 'CH136', 'CH137', 'CH138', 'CH139', 'CH140', 'CH141', 'CH142', 'CH143', 'CH144', 'CH145', 'CH146', 'CH147', 'CH75', 'CH76', 'CH77', 'CH78', 'CH79', 'CH80', 'CH81', 'CH82', 'CH83', 'CH84', 'CH85', 'CH86', 'CH87', 'CH88', 'CH89', 'CH90', 'CH91', 'CH92', 'CH93', 'CH94', 'CH95', 'CH96', 'CH97', 'CH98', 'CH99', 'EKG', 'RA1', 'RA10', 'RA7', 'RA8', 'RA9', 'RH6', 'RH7', 'RH8', 'RHP1', 'RHP2', 'RHP3', 'RHP4', 'RHP5', 'RHP6', 'RPC10']


In [15]:
# From interactive sessions
interactive_annot = bp_filtered_fif.annotations
for x in range(0, len(bp_filtered_fif.annotations)):
    print(interactive_annot[x])
    
    
bp_filtered_fif.annotations.save(f'{raw_data_dir}/ieeg/saved_annotations.fif', overwrite = True) 


OrderedDict([('onset', 77.091797), ('duration', 0.841796875), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 97.789062), ('duration', 2.4453125), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 109.980469), ('duration', 1.65234375), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 112.6875), ('duration', 1.15234375), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 114.886719), ('duration', 49.900390625), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 178.994736), ('duration', 0.121856689453125), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 201.504028), ('duration', 0.1309967041015625), ('description', 'BAD'), ('orig_time', None)])
OrderedDict([('onset', 206.851562), ('duration', 1.796875), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 209.750214), ('duration', 0.0883331298828125), ('description', 'BAD'), ('orig_time', None)])
OrderedDi

/tmp/ipykernel_6093/3057587246.py:7: RuntimeWarning: This filename (/home/brooke/pacman/raw_data/LL19/ieeg/saved_annotations.fif) does not conform to MNE naming conventions. All annotations files should end with -annot.fif, -annot.fif.gz, _annot.fif, _annot.fif.gz, .txt or .csv
  bp_filtered_fif.annotations.save(f'{raw_data_dir}/ieeg/saved_annotations.fif', overwrite = True)


## Epoching the data 

### Onset

In [16]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif...
    Range : 0 ... 784059 =      0.000 ...  1531.365 secs
Ready.
480 events found
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 8193 original time points ...


/tmp/ipykernel_6093/1463149468.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_6093/1463149468.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 240 events and 8193 original time points ...


### Trial End

In [18]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
events = events[0:-1]

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif...
    Range : 0 ... 784059 =      0.000 ...  1531.365 secs
Ready.
480 events found
Event IDs: [0 1]
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 5121 original time points ...
Loading data for 239 events and 5121 original time points ...


/tmp/ipykernel_6093/875914644.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_6093/875914644.py:19: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


### Last Away

In [19]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,1,28569,0,1
1,1,5,32025,0,1
2,2,19,35148,0,1
3,3,6,37836,0,1
4,4,9,39552,0,1
...,...,...,...,...,...
197,233,6,745958,0,1
198,234,19,748595,0,1
199,235,4,752102,0,1
200,236,18,755225,0,1


In [20]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [21]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif...
    Range : 0 ... 784059 =      0.000 ...  1531.365 secs
Ready.
Not setting metadata
202 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 5121 original time points ...
Loading data for 202 events and 5121 original time points ...


/tmp/ipykernel_6093/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_6093/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


### First Dot

In [22]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,1,27929,0,1
1,1,5,31590,0,1
2,2,19,34432,0,1
3,3,6,37427,0,1
4,5,15,41446,0,1
...,...,...,...,...,...
195,234,19,747955,0,1
196,235,4,751539,0,1
197,236,18,754560,0,1
198,237,9,757683,0,1


In [23]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [24]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif...
    Range : 0 ... 784059 =      0.000 ...  1531.365 secs
Ready.
Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 4609 original time points ...
Loading data for 200 events and 4609 original time points ...


/tmp/ipykernel_6093/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_6093/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


## First Move

In [25]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,1,27878,0,1
1,1,5,30924,0,1
2,2,19,34406,0,1
3,3,6,37222,0,1
4,4,9,39552,0,1
...,...,...,...,...,...
233,234,19,747878,0,1
234,235,4,750694,0,1
235,236,18,754380,0,1
236,237,9,757478,0,1


In [26]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [27]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif...
    Range : 0 ... 784059 =      0.000 ...  1531.365 secs
Ready.
Not setting metadata
238 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 5121 original time points ...
Loading data for 238 events and 5121 original time points ...


/tmp/ipykernel_6093/3347570552.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_6093/3347570552.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


## Ghost Attack

In [28]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,1,28518,0,1
1,5,15,43494,0,1
2,14,12,87552,0,1
3,18,11,98124,0,1
4,27,5,125107,0,1
5,37,4,152678,0,1
6,42,8,167884,0,1
7,49,4,184396,0,1
8,53,3,196480,0,1
9,67,6,240179,0,1


In [29]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [30]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif...
    Range : 0 ... 784059 =      0.000 ...  1531.365 secs
Ready.
Not setting metadata
47 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 5121 original time points ...
Loading data for 47 events and 5121 original time points ...


/tmp/ipykernel_6093/2681655913.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_6093/2681655913.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL19/ieeg/LL19_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')
